# Intro. to Snorkel: Extracting Spouse Relations from the News

In this tutorial, we will walk through the process of using `Snorkel` to identify mentions of spouses in a corpus of news articles. The tutorial is broken up into 3 notebooks, each covering a step in the pipeline:
1. Preprocessing
2. Training
3. Evaluation

## Part I: Preprocessing

In this notebook, we preprocess several documents using `Snorkel` utilities, parsing them into a simple hierarchy of component parts of our input data, which we refer to as _contexts_. We'll also create _candidates_ out of these contexts, which are the objects we want to classify, in this case, possible mentions of spouses. Finally, we'll load some gold labels for evaluation.

All of this preprocessed input data is saved to a database.  (Connection strings can be specified by setting the `SNORKELDB` environment variable.  In Snorkel, if no database is specified, then a SQLite database at `./snorkel.db` is created by default--so no setup is needed here!

### Initializing a `SnorkelSession`

First, we initialize a `SnorkelSession`, which manages a connection to a database automatically for us, and will enable us to save intermediate results.  If we don't specify any particular database (see commented-out code below), then it will automatically create a SQLite database in the background for us:

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

# Here, we just set how many documents we'll process for automatic testing- you can safely ignore this!
n_docs = 500 if 'CI' in os.environ else 2591

## Loading the Corpus

Next, we load and pre-process the corpus of documents.

### Configuring a `DocPreprocessor`

We'll start by defining a `TSVDocPreprocessor` class to read in the documents, which are stored in a tab-seperated value format as pairs of document names and text.

In [2]:
from snorkel.parser import TSVDocPreprocessor

doc_preprocessor = TSVDocPreprocessor('data/sample.tsv', max_docs=n_docs)

### Running a `CorpusParser`

We'll use [Spacy](https://spacy.io/), an NLP preprocessing tool, to split our documents into sentences and tokens, and provide named entity annotations.

In [3]:
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser

corpus_parser = CorpusParser(parser=Spacy())
%time corpus_parser.apply(doc_preprocessor, countm=n_docs)

Clearing existing...
Running UDF...
CPU times: user 1.36 s, sys: 22.8 ms, total: 1.38 s
Wall time: 1.48 s


We can then use simple database queries (written in the syntax of [SQLAlchemy](http://www.sqlalchemy.org/), which Snorkel uses) to check how many documents and sentences were parsed:

In [4]:
from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

Documents: 20
Sentences: 770


## Generating Candidates

The next step is to extract _candidates_ from our corpus. A `Candidate` in Snorkel is an object for which we want to make a prediction. In this case, the candidates are pairs of people mentioned in sentences, and our task is to predict which pairs are described as married in the associated text.

### Defining a `Candidate` schema
We now define the schema of the relation mention we want to extract (which is also the schema of the candidates).  This must be a subclass of `Candidate`, and we define it using a helper function. Here we'll define a binary _spouse relation mention_ which connects two `Span` objects of text.  Note that this function will create the table in the database backend if it does not exist:

In [5]:
from snorkel.models import candidate_subclass

#Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
Trend = candidate_subclass('Trend', ['tr'])

### Writing a basic `CandidateExtractor`

Next, we'll write a basic function to extract **candidate spouse relation mentions** from the corpus.  The [Spacy](https://spacy.io/) parser we used performs _named entity recognition_ for us.

We will extract `Candidate` objects of the `Spouse` type by identifying, for each `Sentence`, all pairs of n-grams (up to 7-grams) that were tagged as people. (An n-gram is a span of text made up of n tokens.) We do this with three objects:

* A `ContextSpace` defines the "space" of all candidates we even potentially consider; in this case we use the `Ngrams` subclass, and look for all n-grams up to 7 words long

* A `Matcher` heuristically filters the candidates we use.  In this case, we just use a pre-defined matcher which looks for all n-grams tagged by Spacy as "PERSON". The keyword argument `longest_match_only` means that we'll skip n-grams contained in other n-grams.

* A `CandidateExtractor` combines this all together!

In [6]:
from snorkel.candidates import Ngrams, CandidateExtractor, Noun_chunck
from snorkel.matchers import PersonMatcher, LexiconMatcher

#ngrams         = Ngrams(n_max=7)
#person_matcher = PersonMatcher(longest_match_only=True)
#cand_extractor = CandidateExtractor(Spouse, [ngrams, ngrams], [person_matcher, person_matcher])

# trend_ngrams = Noun_chunck(n_max=1)
trend_ngrams = Ngrams(n_max=1)
trend_matcher = LexiconMatcher(longest_match_only=True)
# trend_extractor = CandidateExtractor(Trend, [trend_ngrams], [trend_matcher])
trend_extractor = CandidateExtractor(Trend, [trend_ngrams], [trend_matcher])

/home/pegah/anaconda3/envs/snorkel/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


Next, we'll split up the documents into train, development, and test splits; and collect the associated sentences.

Note that we'll filter out a few sentences that mention more than five people. These lists are unlikely to contain spouses.

In [7]:
from snorkel.models import Document
from util import number_of_people

docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
#         if number_of_people(s) <= 5:
            if i % 4 == 2:
                dev_sents.add(s)
            elif i % 4 == 3:
                test_sents.add(s)
            else:
                train_sents.add(s)

Finally, we'll apply the candidate extractor to the three sets of sentences. The results will be persisted in the database backend.

In [8]:
%%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    print(i)
   # cand_extractor.apply(sents, split=i)
    #print("Number of candidates:", session.query(Spouse).filter(Spouse.split == i).count())
#     if i == 0:
    trend_extractor.apply(sents, split=i)
    print("Number of (trend) candidates:", session.query(Trend).filter(Trend.split == i).count())
#     if i == 0:
#         x = session.query(Trend).filter(Trend.split == i).all()
#         for e in x:
#             print(x)

 21%|██        | 105/496 [00:00<00:00, 1047.09it/s]

0
Clearing existing...
Running UDF...


  0%|          | 0/129 [00:00<?, ?it/s]

Number of (trend) candidates: 103
1
Clearing existing...
Running UDF...
Number of (trend) candidates: 24
2
Clearing existing...
Running UDF...


100%|██████████| 129/129 [00:00<00:00, 1015.59it/s]

Number of (trend) candidates: 17
CPU times: user 717 ms, sys: 15.7 ms, total: 733 ms
Wall time: 775 ms


## Loading Gold Labels

Finally, we'll load gold labels for development and evaluation. Even though Snorkel is designed to create labels for data, we still use gold labels to evaluate the quality of our models. Fortunately, we need far less labeled data to _evaluate_ a model than to _train_ it.

In [9]:
from my_util import load_external_trend_labels
#, load_external_trend_labels
import time
# time.sleep(12000)

%time missed = load_external_trend_labels(session, Trend, annotator_name='gold')

----------------------------
stable label is found for this one!: 
tr       201601182414.html::span:1768:1773
label                                   -1
Name: 0, dtype: object
----------------------------
stable label is found for this one!: 
tr       201601182414.html::span:3007:3008
label                                   -1
Name: 1, dtype: object
----------------------------
stable label is found for this one!: 
tr       201608040677.html::span:142:143
label                                 -1
Name: 2, dtype: object
----------------------------
stable label is found for this one!: 
tr       201608040677.html::span:271:278
label                                 -1
Name: 3, dtype: object
----------------------------
stable label is found for this one!: 
tr       201608040677.html::span:1595:1602
label                                   -1
Name: 4, dtype: object
----------------------------
stable label is found for this one!: 
tr       201608040677.html::span:2494:2503
label             

----------------------------
stable label is found for this one!: 
tr       201509151162.html::span:3437:3438
label                                   -1
Name: 68, dtype: object
----------------------------
stable label is found for this one!: 
tr       201509151162.html::span:3809:3810
label                                   -1
Name: 69, dtype: object
----------------------------
stable label is found for this one!: 
tr       201509151162.html::span:4801:4809
label                                   -1
Name: 70, dtype: object
----------------------------
stable label is found for this one!: 
tr       201509151162.html::span:5779:5780
label                                   -1
Name: 71, dtype: object
----------------------------
stable label is found for this one!: 
tr       201509151162.html::span:5946:5949
label                                   -1
Name: 72, dtype: object
----------------------------
stable label is found for this one!: 
tr       201509151162.html::span:6546:6551
label

----------------------------
stable label is found for this one!: 
tr       201112140191.html::span:14701:14704
label                                      1
Name: 122, dtype: object
----------------------------
stable label is found for this one!: 
tr       201112140191.html::span:15941:15944
label                                     -1
Name: 123, dtype: object
----------------------------
stable label is found for this one!: 
tr       201112140191.html::span:18251:18260
label                                      1
Name: 124, dtype: object
----------------------------
stable label is found for this one!: 
tr       201112140191.html::span:18295:18300
label                                     -1
Name: 125, dtype: object
----------------------------
stable label is found for this one!: 
tr       201112140191.html::span:19334:19339
label                                     -1
Name: 126, dtype: object
----------------------------
stable label is found for this one!: 
tr       201112140191.h

['201601251979.html::span:335:336']
['201601251979.html::span:587:588']
['201601251979.html::span:1918:1926']
['201601251979.html::span:2232:2240']
['201509151162.html::span:2229:2232']
['201509151162.html::span:2328:2333']
['201509151162.html::span:2459:2462']
['201509151162.html::span:2537:2540']
['201509151162.html::span:2558:2561']
['201509151162.html::span:2857:2863']
['201509151162.html::span:2943:2946']
['201509151162.html::span:3437:3438']
['201509151162.html::span:3809:3810']
['201509151162.html::span:4801:4809']
['201509151162.html::span:5779:5780']
['201509151162.html::span:5946:5949']
['200506200618.html::span:1303:1306']
['200506200618.html::span:1566:1572']
['200506200618.html::span:2405:2406']
['200506200618.html::span:3012:3018']
['200506200618.html::span:4469:4474']
['200506200618.html::span:5471:5472']
['200506200618.html::span:5943:5949']
['200506200618.html::span:6025:6028']
[GoldLabel (gold = -1), GoldLabel (gold = -1), GoldLabel (gold = -1), GoldLabel (gold = -1),

Next, in Part II, we will work towards building a model to predict these labels with high accuracy using data programming